In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets
import torchvision.transforms as transforms
import time

In [2]:
train_model = True

save_path = 'models/'
sub_name = '_cuda'

In [3]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
# print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 11.7
ID of current CUDA device: 0


In [4]:
# Making the code device-agnostic
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Import data

In [5]:
cifar10_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}
data_path = '../dataset/cifar10'
mean = [0.4914, 0.4822, 0.4465] # copied from Q1
std = [0.2470, 0.2435, 0.2616] # copied from Q1
batch_size = 5000

In [6]:
# Load the CIFAR-10 dataset
train_data = datasets.CIFAR10(
        root=data_path,
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    )
cifar10_train = data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


In [7]:
test_data = datasets.CIFAR10(
        root=data_path,
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    )
cifar10_test = data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=True
)

Files already downloaded and verified


# Models For Convolutional Neural Network 

In [8]:
n_channel_in = 3
n_channel_1  = 24
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel1(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        
        # Add a fully connected layer at the end of the CNN
        self.fc = nn.Linear(n_channel_1 * 16 * 16, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc(out)
        return out


In [9]:
n_channel_in = 3
n_channel_1  = 16
n_channel_2  = 32
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel2(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        self.conv2 = nn.Conv2d(n_channel_1, n_channel_2, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        # img is 8 x 8
        
        # Add a fully connected layer at the end of the CNN
        self.fc_nn = nn.Linear(n_channel_2 * 8 * 8, n_out)

    def forward(self, out):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(out)))
        
        out = self.pool2(self.act2(self.conv2(out)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc_nn(out)
        return out


In [10]:
n_channel_in = 3
n_channel_1  = 16
n_channel_2  = 32
n_channel_3  = 64
n_out = len(cifar10_map)

# Define the structure of the CNN
class ConvolutionalModel3(nn.Module):
    def __init__(self):
        super().__init__()
        # img is 32 x 32
        self.conv1 = nn.Conv2d(n_channel_in, n_channel_1, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        # img is 16 x 16
        self.conv2 = nn.Conv2d(n_channel_1, n_channel_2, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        # img is 8 x 8
        self.conv3 = nn.Conv2d(n_channel_2, n_channel_3, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        # img is 4 x 4
        
        # Add a fully connected layer at the end of the CNN
        self.fc_nn = nn.Linear(n_channel_3 * 4 * 4, n_out)

    def forward(self, x):
        # Apply the convolutional filters, activation functions, and pooling layers
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        
        # Flatten the output of the convolutional layers
        out = out.view(out.size(0), -1)
        # Apply the fully connected layer
        out = self.fc_nn(out)
        return out


# Training function

In [11]:
def training(model, training_imgs, loss_fn, optimizer, n_epochs:int, report_period:int = 10):
    sum_time = 0
    for epoch in range(n_epochs):
        start = time.time()
        for images, labels in training_imgs:
            # Convert to device (cuda)
            images = images.to(device)
            labels = labels.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            _outputs = model(images)
            # Compute the loss
            loss = loss_fn(_outputs, labels)
            # Backward pass
            loss.backward()
            # Update the parameters
            optimizer.step()
            
        sum_time += (time.time() - start)
        if (epoch % report_period == 0) or (epoch == n_epochs - 1):
            print(f'epoch {epoch}:\t Loss: {loss:.15f}\t AvgTime: {sum_time/(epoch+1):.2f} s')

# 1 Layer Convolutional Neural Network
Question 2a

In [12]:
model = ConvolutionalModel1()
model = model.to(device)
model 

ConvolutionalModel1(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4096, out_features=10, bias=True)
)

In [13]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)
# optimizer = optimizer.to(device)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)

In [14]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 2.148381471633911	 AvgTime: 18.16 s
epoch 2:	 Loss: 2.012739896774292	 AvgTime: 14.16 s
epoch 4:	 Loss: 1.952702760696411	 AvgTime: 13.54 s
epoch 6:	 Loss: 1.872948884963989	 AvgTime: 13.48 s
epoch 8:	 Loss: 1.836333990097046	 AvgTime: 13.24 s
epoch 10:	 Loss: 1.813726544380188	 AvgTime: 13.15 s
epoch 12:	 Loss: 1.778575420379639	 AvgTime: 13.03 s
epoch 14:	 Loss: 1.759590983390808	 AvgTime: 12.94 s
epoch 16:	 Loss: 1.733090996742249	 AvgTime: 12.85 s
epoch 18:	 Loss: 1.726481080055237	 AvgTime: 12.79 s
epoch 20:	 Loss: 1.687560319900513	 AvgTime: 12.73 s
epoch 22:	 Loss: 1.678763031959534	 AvgTime: 12.70 s
epoch 24:	 Loss: 1.676784753799438	 AvgTime: 12.72 s
epoch 26:	 Loss: 1.645305633544922	 AvgTime: 12.71 s
epoch 28:	 Loss: 1.632562994956970	 AvgTime: 12.68 s
epoch 30:	 Loss: 1.634198546409607	 AvgTime: 12.65 s
epoch 32:	 Loss: 1.591570615768433	 AvgTime: 12.62 s
epoch 34:	 Loss: 1.601046442985535	 AvgTime: 12.61 s
epoch 36:	 Loss: 1.589016675949097	 AvgTime: 12.59 

In [15]:
if train_model :
    torch.save(model.state_dict(), save_path + f'1_layer_cnn{sub_name}.pt')

In [12]:
load_model = ConvolutionalModel1()
load_model = load_model.to(device)
load_model.load_state_dict(torch.load(save_path + f'1_layer_cnn{sub_name}.pt'))

<All keys matched successfully>

In [13]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.62388


In [14]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.58070


# 2 Layer Convolutional Neural Network
Question 2b

In [19]:
model = ConvolutionalModel2()
model = model.to(device)
model 

ConvolutionalModel2(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_nn): Linear(in_features=2048, out_features=10, bias=True)
)

In [20]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)

In [21]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 2.286010026931763	 AvgTime: 12.31 s
epoch 2:	 Loss: 2.252520799636841	 AvgTime: 12.14 s
epoch 4:	 Loss: 2.213654756546021	 AvgTime: 12.20 s
epoch 6:	 Loss: 2.172257900238037	 AvgTime: 12.19 s
epoch 8:	 Loss: 2.127006053924561	 AvgTime: 12.19 s
epoch 10:	 Loss: 2.072481632232666	 AvgTime: 12.15 s
epoch 12:	 Loss: 2.032099008560181	 AvgTime: 12.12 s
epoch 14:	 Loss: 1.996951699256897	 AvgTime: 12.15 s
epoch 16:	 Loss: 1.967066168785095	 AvgTime: 12.16 s
epoch 18:	 Loss: 1.937609434127808	 AvgTime: 12.13 s
epoch 20:	 Loss: 1.915834546089172	 AvgTime: 12.13 s
epoch 22:	 Loss: 1.876950025558472	 AvgTime: 12.16 s
epoch 24:	 Loss: 1.851774573326111	 AvgTime: 12.18 s
epoch 26:	 Loss: 1.833824753761292	 AvgTime: 12.18 s
epoch 28:	 Loss: 1.828540444374084	 AvgTime: 12.17 s
epoch 30:	 Loss: 1.814255237579346	 AvgTime: 12.17 s
epoch 32:	 Loss: 1.798223018646240	 AvgTime: 12.17 s
epoch 34:	 Loss: 1.771083354949951	 AvgTime: 12.19 s
epoch 36:	 Loss: 1.754318118095398	 AvgTime: 12.18 

In [22]:
if train_model :
    torch.save(model.state_dict(), save_path + f'2_layer_cnn{sub_name}.pt')

In [15]:
load_model = ConvolutionalModel2()
load_model = load_model.to(device)
load_model.load_state_dict(torch.load(save_path + f'2_layer_cnn{sub_name}.pt'))

<All keys matched successfully>

In [16]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.59416


In [17]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.57520


# 3 Layer Convolutional Neural Network
extra

In [18]:
model = ConvolutionalModel3()
model = model.to(device)
model 

ConvolutionalModel3(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_nn): Linear(in_features=1024, out_features=10, bias=True)
)

In [19]:
learn_rate = 1e-2
report_rate = 2
n_epochs = 300

optimizer = optim.SGD(model.parameters(), lr=learn_rate)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)

In [20]:
if train_model:
    training(model, 
         cifar10_train, 
         loss, 
         optimizer, 
         n_epochs,
         report_period=report_rate
         )

epoch 0:	 Loss: 2.304883241653442	 AvgTime: 18.19 s
epoch 2:	 Loss: 2.297615766525269	 AvgTime: 17.76 s
epoch 4:	 Loss: 2.288931131362915	 AvgTime: 17.28 s
epoch 6:	 Loss: 2.280941009521484	 AvgTime: 17.60 s
epoch 8:	 Loss: 2.275190114974976	 AvgTime: 17.46 s
epoch 10:	 Loss: 2.261464118957520	 AvgTime: 17.62 s
epoch 12:	 Loss: 2.240642309188843	 AvgTime: 17.40 s
epoch 14:	 Loss: 2.220862150192261	 AvgTime: 17.42 s
epoch 16:	 Loss: 2.194751977920532	 AvgTime: 17.37 s
epoch 18:	 Loss: 2.162788152694702	 AvgTime: 17.30 s
epoch 20:	 Loss: 2.136578559875488	 AvgTime: 17.21 s
epoch 22:	 Loss: 2.098626852035522	 AvgTime: 17.12 s
epoch 24:	 Loss: 2.069032907485962	 AvgTime: 17.22 s
epoch 26:	 Loss: 2.043383836746216	 AvgTime: 17.14 s
epoch 28:	 Loss: 2.017389059066772	 AvgTime: 17.08 s
epoch 30:	 Loss: 2.000645160675049	 AvgTime: 17.01 s
epoch 32:	 Loss: 1.983270525932312	 AvgTime: 16.95 s
epoch 34:	 Loss: 1.966071248054504	 AvgTime: 16.96 s
epoch 36:	 Loss: 1.947727203369141	 AvgTime: 16.99 

In [21]:
if train_model :
    torch.save(model.state_dict(), save_path + f'3_layer_cnn{sub_name}.pt')

In [22]:
load_model = ConvolutionalModel3()
load_model = load_model.to(device)
load_model.load_state_dict(torch.load(save_path + f'3_layer_cnn{sub_name}.pt'))

<All keys matched successfully>

In [23]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_train:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 50000
Accuracy: 0.58290


In [24]:
# Evaluate the CNN on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cifar10_test:
        # convert to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = load_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Report the training time, training loss, and evaluation accuracy
print(f'Total: {total}')
print(f'Accuracy: {correct / total:.5f}')

Total: 10000
Accuracy: 0.57280
